In [6]:
from common import *

cursor = connect()

# BEFORE UPDATE

语法：
```
CREATE TRIGGER trigger_name
BEFORE UPDATE
ON table_name
FOR EACH {ROW | STATEMENT}
EXECUTE FUNCTION trigger_function();
```

In [2]:
sql = """
CREATE TABLE employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    salary NUMERIC NOT NULL
);

CREATE OR REPLACE FUNCTION fn_before_update_salary()
RETURNS TRIGGER AS $$
BEGIN
    IF NEW.salary < OLD.salary THEN
        RAISE EXCEPTION 'New salary cannot be less than current salary';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER before_update_salary_trigger
BEFORE UPDATE OF salary ON employees
FOR EACH ROW
EXECUTE FUNCTION fn_before_update_salary();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x24781109550>

In [3]:
sql = """
INSERT INTO employees(name, salary)
VALUES
   ('John Doe', 70000),
   ('Jane Doe', 80000)
RETURNING *;
"""
run_sql(cursor, sql)

   id      name salary
0   1  John Doe  70000
1   2  Jane Doe  80000


In [4]:
sql = """
UPDATE employees
SET salary = salary * 0.9
WHERE id = 1;
"""
run_sql(cursor, sql)

RaiseException: New salary cannot be less than current salary
CONTEXT:  在RAISE的第4行的PL/pgSQL函数fn_before_update_salary()

# AFTER UPDATE

语法：
```
CREATE TRIGGER trigger_name
AFTER UPDATE
ON table_name
FOR EACH {ROW | STATEMENT}
EXECUTE FUNCTION trigger_function();
```

In [7]:
sql = """
CREATE TABLE salaries(
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    salary NUMERIC NOT NULL
);

CREATE TABLE salary_changes (
    id SERIAL PRIMARY KEY,
    employee_id INT NOT NULL,
    old_salary NUMERIC NOT NULL,
    new_salary NUMERIC NOT NULL,
    changed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE OR REPLACE FUNCTION log_salary_change()
RETURNS TRIGGER 
AS 
$$
BEGIN
    INSERT INTO salary_changes (employee_id, old_salary, new_salary)
    VALUES (NEW.id, OLD.salary, NEW.salary);
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER after_update_salary_trigger
AFTER UPDATE OF salary ON salaries
FOR EACH ROW
EXECUTE FUNCTION log_salary_change();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x24782dd1910>

In [8]:
sql = """
INSERT INTO salaries(name, salary)
VALUES
   ('John Doe', 90000),
   ('Jane Doe', 95000)
RETURNING *;
"""
run_sql(cursor, sql)

   id      name salary
0   1  John Doe  90000
1   2  Jane Doe  95000


In [11]:
sql = """
UPDATE salaries
SET salary = salary * 1.05
WHERE id = 1
RETURNING *;
"""
run_sql(cursor, sql)

   id      name      salary
0   1  John Doe  99225.0000


In [10]:
sql = """
SELECT * FROM salary_changes;
"""
run_sql(cursor, sql)

   id  employee_id old_salary new_salary                 changed_at
0   1            1      90000   94500.00 2024-04-17 08:13:51.540730
